<a href="https://colab.research.google.com/github/xerographica/xerographica.github.io/blob/master/Diffusers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 参照元 https://yuuyuublog.org/diffusers/
#stable diffusionのインストール
!pip install --upgrade diffusers[torch] transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 24.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 104.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 29.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.6/227.6 kB 30.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 93.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 55.7 MB/s eta 0:00:00


In [ ]:
# これはGoogle Driveをマウントするためのコードです。
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#@title
# ここでは、LoRAを読み込む際に使用する「load_safetensors_lora」という関数を定義してます。
!pip install safetensors

import torch
from safetensors.torch import load_file

def load_safetensors_lora(pipeline, checkpoint_path, LORA_PREFIX_UNET="lora_unet", LORA_PREFIX_TEXT_ENCODER="lora_te", alpha=0.75):
# from https://qiita.com/Limitex/items/275d91dd4acdbf57b5f4
#   and from https://github.com/huggingface/diffusers/blob/main/scripts/convert_lora_safetensor_to_diffusers.py
#########################################################
    # load LoRA weight from .safetensors
    state_dict = load_file(checkpoint_path)

    visited = []

    # directly update weight in diffusers model
    for key in state_dict:
        # it is suggested to print out the key, it usually will be something like below
        # "lora_te_text_model_encoder_layers_0_self_attn_k_proj.lora_down.weight"

        # as we have set the alpha beforehand, so just skip
        if ".alpha" in key or key in visited:
            continue

        if "text" in key:
            layer_infos = key.split(".")[0].split(LORA_PREFIX_TEXT_ENCODER + "_")[-1].split("_")
            curr_layer = pipeline.text_encoder
        else:
            layer_infos = key.split(".")[0].split(LORA_PREFIX_UNET + "_")[-1].split("_")
            curr_layer = pipeline.unet

        # find the target layer
        temp_name = layer_infos.pop(0)
        while len(layer_infos) > -1:
            try:
                curr_layer = curr_layer.__getattr__(temp_name)
                if len(layer_infos) > 0:
                    temp_name = layer_infos.pop(0)
                elif len(layer_infos) == 0:
                    break
            except Exception:
                if len(temp_name) > 0:
                    temp_name += "_" + layer_infos.pop(0)
                else:
                    temp_name = layer_infos.pop(0)

        pair_keys = []
        if "lora_down" in key:
            pair_keys.append(key.replace("lora_down", "lora_up"))
            pair_keys.append(key)
        else:
            pair_keys.append(key)
            pair_keys.append(key.replace("lora_up", "lora_down"))

        # update weight
        if len(state_dict[pair_keys[0]].shape) == 4:
            weight_up = state_dict[pair_keys[0]].squeeze(3).squeeze(2).to(torch.float32)
            weight_down = state_dict[pair_keys[1]].squeeze(3).squeeze(2).to(torch.float32)
            curr_layer.weight.data += alpha * torch.mm(weight_up, weight_down).unsqueeze(2).unsqueeze(3)
        else:
            weight_up = state_dict[pair_keys[0]].to(torch.float32)
            weight_down = state_dict[pair_keys[1]].to(torch.float32)
            curr_layer.weight.data += alpha * torch.mm(weight_up, weight_down)

        # update visited list
        for item in pair_keys:
            visited.append(item)

    return pipeline
#########################################################

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from diffusers import StableDiffusionPipeline, EulerDiscreteScheduler
from diffusers.models import AutoencoderKL
import torch

#画像生成に使うモデルデータ
model_id = "AIARTCHAN/AbyssHellVer3"#@param {type:"string"}
# model_id = "AIARTCHAN/AbyssHellVer3"#@param {type:"string"}
#画像生成に使うVAE
vae = "stabilityai/sd-vae-ft-ema" #@param {type:"string"} ## default stabilityai/sd-vae-ft-ema
vae = AutoencoderKL.from_pretrained(vae)

#画像生成に使うスケジューラー
scheduler = EulerDiscreteScheduler.from_pretrained(model_id, subfolder="scheduler")

#パイプラインの作成
pipe = StableDiffusionPipeline.from_pretrained(model_id, scheduler=scheduler, vae=vae, custom_pipeline="lpw_stable_diffusion")

#LoRAを読み込む
LoRA_USE = True #@param {type:"boolean"}
if LoRA_USE == True:
  LoRA="/content/drive/MyDrive/Diffusers/LoRA/flat2.safetensors"#@param {type:"string"}
#  LoRA="/content/drive/MyDrive/Diffusers/LoRA/Pepper-LoRA.safetensors"#@param {type:"string"}
#  LoRA="/content/drive/MyDrive/Diffusers/LoRA/flat1.safetensors"#@param {type:"string"}
  LoRA_alpha = -1#@param {type:"number"}
  pipe = load_safetensors_lora(pipe, LoRA, alpha=LoRA_alpha)

pipe = pipe.to("cuda")

#NSFW規制を無効化する
if pipe.safety_checker is not None:
  pipe.safety_checker = lambda images, **kwargs: (images, False)

text_encoder/model.safetensors not found


In [ ]:
import random
import datetime
import os

#txt2img出力画像の保存先
!mkdir -p /content/txt2img_output

# ファイル名に使う日付と時刻のフォーマットを定義する
file_format = "%Y%m%d_%H%M%S"
i=0

#ポジティブプロンプト
prompt = "1 lady, ((very wide shot)), black hair, black eyes, sweater, leggins, snowing, street, blash, small breasts, standing, light smile, looking at viewer, highres, absurdres, exquisite, beautiful, masterpiece, 8K, best quality, high resolution, "#@param {type:"string"}
# laplace prompt "laplace, HAIR ORNAMENT, WHITE COAT, BELT, BLACK SWIMSUIT, FUR TRIM, GLOVES, BLUE BOOTS, white background, masterpiece,1 lady, (full body), standing, salute, snap to attention, laughing, looking at viewer"
# "white skin, masterpiece,1 lady, black hair, standing, shiny skin, slender, posing, blush, light smile, looking at viewer, futuristic city, summer, sunny day, odd eyes, photorealistic, swimsuit, hairband, small breasts, two tone hair"

#ネガティブプロンプト
n_prompt = "(worst quality,low quality:1.4),(monochrome,greyscale:1.4), bad anatomy, bad hands, nipples, bukeh, blurry, lowres,  signature, text, watermark, long neck, long body"#@param {type:"string"}
#CFG Scale
CFG_scale = 7 #@param {type:"number"}
#ステップ数
Steps = 20 #@param {type:"number"}
#seed値
seed=-1 #@param {type:"number"}
if seed == -1:
  seed = random.randint(0, 2147483647)
  generator = torch.Generator(device="cuda").manual_seed(seed)
else:
  generator = torch.Generator(device="cuda").manual_seed(seed)
#生成枚数
num_images = 1 #@param {type:"number"}
#出力画像の横幅
width = 768 #@param {type:"number"}
#出力画像の高さ
height = 512 #@param {type:"number"}
#出力画像を保存するフォルダ
save_path = "/content/txt2img_output" #@param {type:"string"}
# ↑ default : /content/txt2img_output
# not worked : /content/drive/MyDrive/Diffusers/Output

while i < int(num_images):
  #画像を生成
  image = pipe(prompt, negative_prompt=n_prompt, width=width, height=height, generator=generator, guidance_scale=CFG_scale, num_inference_steps=Steps).images[0]

  # 現在の日本時間を取得
  jst_now = datetime.datetime.now(datetime.timezone(datetime.timedelta(hours=9)))
  #出力する画像の名前を生成する
  seedCount = seed + i
  file_name = (jst_now.strftime(file_format)+ "_" + str(seedCount))
  image_name = file_name + f".png"

  #画像を保存する
  save_location = os.path.join(save_path, image_name)
  image.save(save_location)
  i = i + 1


# ファイルをDL
from google.colab import files
import shutil
# フォルダをzip圧縮
shutil.make_archive("txt2img_output", "zip", "/content/txt2img_output")
# shutil.make_archive("txt2img_output", "zip", "/content/drive/MyDrive/Diffusers/Output")
# 圧縮ファイルをダウンロード
files.download("txt2img_output.zip")

import shutil
# /content/txt2img_outputフォルダ内のデータを削除
# shutil.rmtree("/content/drive/MyDrive/Diffusers/Output")
shutil.rmtree("/content/txt2img_output")


  0%|          | 0/20 [00:00<?, ?it/s]

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# ファイルをDL
from google.colab import files
import shutil
# フォルダをzip圧縮
shutil.make_archive("txt2img_output", "zip", "/content/txt2img_output")
# shutil.make_archive("txt2img_output", "zip", "/content/drive/MyDrive/Diffusers/Output")
# 圧縮ファイルをダウンロード
files.download("txt2img_output.zip")

In [ ]:
import shutil
# /content/txt2img_outputフォルダ内のデータを削除
# shutil.rmtree("/content/drive/MyDrive/Diffusers/Output")
shutil.rmtree("/content/txt2img_output")

In [ ]:
# (まだうまくいっていない) Civitai変換スクリプト
# 格納するフォルダを作成
import os

!mkdir -p /content/script

#Driveから変換スクリプトをコピー
!cp /content/drive/MyDrive/Diffusers/Scripts/convert_diffusers20_original_sd.py /content/script
!cp /content/drive/MyDrive/Diffusers/Scripts/model_util.py /content/script

In [ ]:
%cd /content/script

#モデルデータを変換する
!python convert_diffusers20_original_sd.py /content/drive/MyDrive/Diffusers/Model/chilledremix\chilled_remix_v2.safetensors /content/drive/MyDrive/Diffusers/Model/chilledremix --v1 --reference_model runwayml/stable-diffusion-v1-5

/content/script
2023-06-18 04:28:30.256831: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
loading Diffusers: /content/drive/MyDrive/Diffusers/Model/chilledremixchilled_remix_v2.safetensors
╭───────────────────── Traceback (most recent call last) ──────────────────────╮
│ /content/script/convert_diffusers20_original_sd.py:93 in <module>            │
│                                                                              │
│   90 │   │   │   │   │     help="model to save: checkpoint (with extension)  │
│   91                                                                         │
│   92   args = parser.parse_args()                                            │
│ ❱ 93   convert(args)                                                         │
│   94                                                                         │
│                                                                              │
│ /content/script/convert_diffus